In [9]:
import pandas as pd

pd.set_option('display.max_columns', None)

In [10]:
df = pd.read_csv(r'C:\Users\igors\Downloads\_data.csv')

In [11]:
df.head(1)

,Unnamed: 0,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Телефоны,Описание,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление
0,0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157


## Количество комнат

In [12]:
# Для заполнения пропущенных значений в столбце "Количество комнат" нам потребуется создать временные столбцы


# На сайте видно, что на первом месте заспличенного столбца "Площадь, м2" всегда общая площадь
df['total_area'] = df['Площадь, м2'].apply(lambda x: float(x.split('/')[0]))

# Заменим NaN на строковые значения, чтобы затем создать еще один временный столбец с количеством комнат, куда будем складывать результаты вычислений
df['Количество комнат'].fillna('Unknown', inplace=True)

df['rooms_num'] = df['Количество комнат'].apply(lambda x: int(x.split(',')[0]) if x != 'Unknown' else x)

# В некоторых строках с незаполненным количеством комнат, указаны площади комнат. Посчитаем количество площадей - это и будет количеством комнат
df['area_based_room_num'] = df['Площадь комнат, м2'].apply(lambda x: len(x.split()) if isinstance(x, str) else x)

# Создадим фильтр и занесем посчиатнные значения во временный столбец
mask = (df['rooms_num'] == 'Unknown') & (~df['area_based_room_num'].isna())
df.loc[mask, 'rooms_num'] = df.loc[mask, 'area_based_room_num']

# Находим строки, где значение столбца 'rooms_num' неизвестно, а в 'Описании' упоминается "студия" или "однокомнатность",
# и присваиваем им значение 1 в столбце 'rooms_num'
df.loc[(df['rooms_num'] == 'Unknown') & (df['Описание'].str.contains('студи|однок', case=False)), 'rooms_num'] = 1

In [15]:
df['rooms_num'].value_counts()

rooms_num
1          8796
2          8466
3          4262
4          1163
5           367
Unknown     176
6           138
Name: count, dtype: int64

In [17]:
# Осталось совсем немного неизвестных значений, поэтому оставшиеся заполним опираясь на медианную площадь
df[df['rooms_num'] != 'Unknown'][['rooms_num', 'total_area']].groupby('rooms_num').agg({'total_area': 'median'})

,total_area
rooms_num,
1,37.0
2,52.0
3,80.0
4,145.0
5,205.0
6,328.5


In [18]:
df.loc[(df['rooms_num'] == 'Unknown') & (df['total_area'] < 52), 'rooms_num'] = 1
df.loc[(df['rooms_num'] == 'Unknown') & ~(df['total_area'] < 52), 'rooms_num'] = 2

In [19]:
# Теперь заполним пропущенные значения в столбец Количество комнат

df.loc[df['Количество комнат'] == 'Unknown', 'Количество комнат'] = df.loc[df['Количество комнат'] == 'Unknown', 'rooms_num']

## Метро

In [20]:
# Применим тот же подход, что и на предыдущем шаге

df['Метро'].fillna('Unknown', inplace=True)

df['metro'] = df['Метро'].apply(lambda x: x.split('(')[0])

In [27]:
# При изучении датафрейма оказалось, что большое кол-во пропущенных станций метро приходятся на Зеленоград. Заменим их на 
# ближайшее метро
df.loc[(df['metro'] == 'Unknown') & (df['Адрес'].str.contains('зеленогр', case=False)), 'metro'] = 'Ховрино (25 мин на машине)'

## Парковка

In [29]:
# Применим тот же подход, что и на предыдущем шаге. К третьему релизу проанализируем влияет ли наличие информации о парковке 
# на стоимость аренды
df['Парковка'].fillna('Unknown', inplace=True)

### в остальных столбцах пропущенных значений нет

In [33]:
# Создаем словарь с соответствием текущих и новых имен столбцов
rename_dict = {'ID  объявления': 'id', 
               'Количество комнат': 'rooms',
               'Тип': 'type',
               'Метро': 'metro',
               'Адрес': 'adress',
               'Площадь, м2': 'area',
               'Дом': 'building',
               'Парковка': 'parking'}

# Используем метод rename() с передачей словаря в параметр columns
df.rename(columns=rename_dict, inplace=True)

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23368 entries, 0 to 23367
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                23368 non-null  int64  
 1   id                        23368 non-null  int64  
 2   rooms                     23368 non-null  object 
 3   type                      23368 non-null  object 
 4   metro                     23368 non-null  object 
 5   adress                    23368 non-null  object 
 6   area                      23368 non-null  object 
 7   building                  23368 non-null  object 
 8   parking                   23368 non-null  object 
 9   Цена                      23368 non-null  object 
 10  Телефоны                  23368 non-null  object 
 11  Описание                  23368 non-null  object 
 12  Ремонт                    20613 non-null  object 
 13  Площадь комнат, м2        14458 non-null  object 
 14  Балкон